In [9]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="7"

import tqdm
import h5py
import clip
import torch
from torch.utils.data import DataLoader
from dataloader_video import VideoCLIPDataset

import math
import urllib.request
import clip
import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image
from torchvision import transforms
import cv2

#Set the data directory
data_dir = '../../data/'


In [10]:

frame_num = 32
dataset = VideoCLIPDataset(None, frame_num, os.path.join(data_dir, "videos/*.mp4"))
print(len(dataset))

dataloader = DataLoader(
    dataset,
    batch_size=1,
    num_workers = 16,
    shuffle=False
)
data_iter = iter(dataloader)

40


In [13]:
# Load CLIP model.
clip_model = "ViT-B/32" #@param ["RN50", "RN101", "RN50x4", "RN50x16", "ViT-B/32", "ViT-B/16"]
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load(clip_model, device=device, jit=False)

100%|███████████████████████████████████████| 338M/338M [00:16<00:00, 21.2MiB/s]


## Extract Feature

In [16]:
os.makedirs(os.path.join(data_dir, 'frame_feat'))
dataset_feats = h5py.File(os.path.join(data_dir, "frame_feat/clip_patch_feat_all.h5"), "w")
dataset_feats.create_dataset("features", (len(dataset), 32, 17, 512))
dataset_feats.create_dataset("ids", (len(dataset), ), 'S50')
# dataset_feats.close()

<HDF5 dataset "ids": shape (40,), type "|S50">

In [17]:
global_index = 0
video_ids = {}
data_iter = iter(dataloader)
for batch in tqdm.tqdm(data_iter):
    batch_size = batch['video'].shape[0]
    for i in range(batch_size):
        for j in range(frame_num):
            with torch.no_grad():  
                image_features = model.encode_image(batch['video'][i][j].cuda())
            dataset_feats['features'][global_index, j] = image_features.detach().cpu().numpy()
        dataset_feats['ids'][global_index] = batch['vid'][i].split('/')[-1].strip('.mp4').encode("ascii", "ignore")  #b'../data_582/videos/bathroom_cabinet_2.mp4' =>'bathroom_cabinet_2'
        print(dataset_feats['ids'][global_index])
        global_index += 1
    



  2%|▎         | 1/40 [00:05<03:23,  5.21s/it]

b'using_computer_2'


  5%|▌         | 2/40 [00:05<01:28,  2.34s/it]

b'watch_tv_1'


  8%|▊         | 3/40 [00:05<00:51,  1.40s/it]

...

 98%|█████████▊| 39/40 [00:18<00:00,  3.78it/s]

b'using_oven_0'


100%|██████████| 40/40 [00:18<00:00,  3.84it/s]

b'bathroom_cabinet_2'


100%|██████████| 40/40 [00:18<00:00,  2.16it/s]


In [18]:
#Close the file pointer
dataset_feats.close()

# Check the extracted feature


In [19]:
import os.path as osp
import h5py
data_path = '../data'
encoding = 'utf-8'
frame_feats = {}
app_feat_file = osp.join(data_dir, f'frame_feat/clip_patch_feat_all.h5')
with h5py.File(app_feat_file, 'r') as fp:
        vids = fp['ids']
        feats = fp['features']
        print(feats.shape)  # v_num, clip_num, feat_dim
        for id, (vid, feat) in enumerate(zip(vids, feats)):
            vid = vid.decode(encoding)
            frame_feats[vid] = feat

fp.close()
frame_feats

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '../data_582/frame_feat/clip_patch_feat_all.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## Check Results

In [ ]:
batch = data_iter.next()

In [ ]:
# Load CLIP model.
clip_model = "ViT-B/32" #@param ["RN50", "RN101", "RN50x4", "RN50x16", "ViT-B/32", "ViT-B/16"]
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load(clip_model, device=device, jit=False)

In [ ]:
def load_image(raw_img, resize=None, pil=False):
    if isinstance(raw_img, str):
        image = Image.open(raw_img).convert("RGB")
    else:
        image = Image.fromarray(raw_img).convert("RGB")
    
    if resize is not None:
        image = image.resize((resize, resize))
    return np.asarray(image).astype(np.float32) / 255.

def viz_patches(x, figsize=(20, 20), patch_idx=None, topk=None, t=5):
    # x: num_patches, 3, patch_size, patch_size
    n = x.shape[0]
    nrows = int(math.sqrt(n))
    _, axes = plt.subplots(nrows, nrows, figsize=figsize)
    for i, ax in enumerate(axes.flatten()):            
        im = x[i].permute(1, 2, 0).numpy()
        im = (im * 255.).round().astype(np.uint8)
        if patch_idx is not None and i == patch_idx:
            im[0:t] = (255, 0, 0)
            im[im.shape[0]-t:] = (255, 0, 0)
            im[:, 0:t] = (255, 0, 0)
            im[:, im.shape[1]-t:] = (255, 0, 0)
        if topk is not None:
            if i in topk and i != patch_idx:
                im[0:t] = (255, 255, 0)
                im[im.shape[0]-t:] = (255, 255, 0)
                im[:, 0:t] = (255, 255, 0)
                im[:, im.shape[1]-t:] = (255, 255, 0)
        ax.imshow(im)
        ax.axis("off")
    plt.show()

def patchify(image_path, resolution, patch_size, patch_stride=None):
    img_tensor = transforms.ToTensor()(load_image(image_path, resolution, True))
    if patch_stride is None:
        patch_stride = patch_size
    patches = img_tensor.unfold(
        1, patch_size, patch_stride).unfold(2, patch_size, patch_stride)
    patches = patches.reshape(3, -1, patch_size, patch_size).permute(1, 0, 2, 3)
    return patches  # N, 3, patch_size, patch_size

In [ ]:
image_resolution = 224 * 4
patch_size = 224
image_index = 0
one_frame = batch['raw_frame'][0][image_index].permute(1,2,0).numpy()
raw_patches = patchify(one_frame, image_resolution, patch_size, patch_stride=None)
print("patches: ", raw_patches.shape)
viz_patches(raw_patches, figsize=(8, 8))

In [ ]:
plt.imshow((batch['video'][0][image_index][0].permute(1, 2, 0).numpy() * 255.).round().astype(np.uint8))
plt.show()

In [ ]:
#@title Detect
patches = batch['video'][0][image_index][1:]

clip_model = "ViT-B/32" #@param ["RN50", "RN101", "RN50x4", "RN50x16", "ViT-B/32", "ViT-B/16"]
image_caption = 'green flower' #@param {type:"string"}
topk =  4#@param {type:"integer"}

text_input = clip.tokenize([image_caption]).to(device)

patches_pad = patches.to(device)

with torch.no_grad():
    patch_embs = model.encode_image(patches_pad)
    text_embs = model.encode_text(text_input)
    patch_embs = patch_embs / patch_embs.norm(dim=-1, keepdim=True)
    text_embs = text_embs / text_embs.norm(dim=-1, keepdim=True)
    sim = patch_embs @ text_embs.t()
    idx_max = sim.argmax().item()
    topk_idxs = torch.topk(sim.flatten(), topk)[-1].cpu().numpy().tolist()

viz_patches(patches, figsize=(10, 10), patch_idx=idx_max, topk=topk_idxs, t=int(0.05*patch_size))

In [ ]:
#@title Detect
patches = patchify(one_frame, image_resolution, patch_size, patch_stride=None)


# image_caption = 'hand' #@param {type:"string"}
topk =  4#@param {type:"integer"}

text_input = clip.tokenize([image_caption]).to(device)

patches_pad = patches.to(device)

with torch.no_grad():
    patch_embs = model.encode_image(patches_pad)
    text_embs = model.encode_text(text_input)
    patch_embs = patch_embs / patch_embs.norm(dim=-1, keepdim=True)
    text_embs = text_embs / text_embs.norm(dim=-1, keepdim=True)
    sim = patch_embs @ text_embs.t()
    idx_max = sim.argmax().item()
    topk_idxs = torch.topk(sim.flatten(), topk)[-1].cpu().numpy().tolist()

viz_patches(raw_patches, figsize=(10, 10), patch_idx=idx_max, topk=topk_idxs, t=int(0.05*patch_size))